In [92]:
from scipy.optimize import minimize
import numpy as np

## Optimize the radius and centers of the hyper-spheres

Let the given graph(network) is represented by $G=(V,E)$. For each vertex $u \in V(G)$, we associate a hyper-sphere of radius $R_u$ and center at $C_u$. Let us denote the set of radius as $R=\{R_u: u \in V(G)\}$ and $C = \{C_u: u \in V(G)\}$. If the hyper-sphere is $K$-dimensional, then $R \in {\rm I\!R}^{|V|}$ and $C \in {\rm I\!R}^{|V| \times K}$.

Let $C_u$ and $R_u$ and $C_v$ and $R_v$ be the center and radius of two $K$-dimensional hyper-spheres corresponding to vertices $u, v$ respectively. $C_u, C_v \in {\rm I\!R}^K$ and $R_u, R_v \in {\rm I\!R}$. Let $X_{uv}$ be the embedding of the edge $e=(u,v) \in E(G)$. Let us denote the embedding set as $X=\{X_{uv}: (u,v) \in E\}$.

The optimization problem we are trying to solve here is as follows:
<br>

Minimize &nbsp; $\alpha \sum_{u \in V}{R_u^2}$ <br>
subject to $\left\Vert X_{uv} - C_{u} \right\Vert^2 \leq R_u^2$  and  $\left\Vert X_{uv} - C_{v} \right\Vert^2 \leq R_v^2 \quad \forall (u,v) \in E$
            
The optimization variables in this problem are $R, C$.



In [261]:
def _constraint(params, args):
    X_uv = args[0]
    embed_dim = len(X_uv)
    param_index = args[1]
    r = params[param_index*(embed_dim+1)]
    c = params[param_index*(embed_dim+1)+1:(param_index+1)*(embed_dim+1)]
    val = r**2 - np.linalg.norm(np.array(X_uv) - np.array(c))**2
    return val

def _func_objective(params, args):
    embed_dim = len(list(args[0].values())[0])
    rad_param_count = int(len(params)/(embed_dim+1))
    total_rad_sqr = 0
    for index in range(rad_param_count):
        r = params[index*(embed_dim+1)]
        total_rad_sqr += r**2
    return total_rad_sqr

def _minimize_projection_loss(init_params, args):
    constraints = []
    for key in args[0].keys():
        u = key[0]
        v = key[1]
        X_uv = args[0][key]
        args_1 = [X_uv, u]
        args_2 = [X_uv, v]
        constraint_1 = {'type':'ineq', 'fun': _constraint, 'args':[args_1]}
        constraint_2 = {'type':'ineq', 'fun': _constraint, 'args':[args_2]}
        constraints.append(constraint_1)
        constraints.append(constraint_2)
    
    params = minimize(_func_objective, [init_params],
                      args=args,
                      constraints=constraints)
    return params['x']

def optimize_collective_homophily(embeddings, centers, radii, edge_map, nodes):
    args = {}
    for edge_index, edge in edge_map.items():
        node_u = edge[0]
        node_v = edge[1]
        node_u_ind = np.where(nodes == node_u)[0][0]
        node_v_ind = np.where(nodes == node_v)[0][0]
        args[(node_u_ind, node_v_ind)] = embeddings[edge_index]
    
    embed_dim = centers.shape[1]
    node_count = len(nodes)
    assert node_count == centers.shape[0]
    init_params = np.empty((node_count, embed_dim + 1))
    for i in range(node_count):
        c_i = list(centers[i])
        r_i = list(radii[i])
        init_params[i] = r_i + c_i
    opt_params = _minimize_projection_loss(init_params, [args])
    print(opt_params.shape)
    
    opt_centers = np.empty((node_count, embed_dim))
    opt_radii = np.empty((node_count, 1))
    for i in range(node_count):
        opt_radii[i] = opt_params[i*(embed_dim+1)]
        opt_centers[i] = opt_params[i*(embed_dim+1)+1:(i+1)*(embed_dim+1)]
    return opt_centers, opt_radii

### Test data for the above optimization problem

In [262]:
# Test network - I
embeddings = np.array([[2, 0], [6, 0], [10, 0]])
centers = np.array([[0, 0], [4, 0], [8, 0], [12, 0]])
radii = np.array([[3], [3], [3], [3]])
edge_map = {0:(1, 2), 1:(2, 3), 2:(3, 4)}
nodes = np.array([1, 2, 3, 4])

# Test network - II
# embeddings = np.array([[2, 0]])
# centers = np.array([[0, 0], [4, 0]])
# radii = np.array([[3], [3]])
# edge_map = {0:(1, 2)}
# nodes = np.array([1, 2])

optimize_collective_homophily(embeddings, centers, radii, edge_map, nodes)

(12,)
[[ 1.99999982e+00 -3.06836096e-08]
 [ 4.00000000e+00  0.00000000e+00]
 [ 8.00000000e+00  0.00000000e+00]
 [ 9.99999976e+00 -4.07633895e-08]]
[[4.79916380e-04]
 [2.00000000e+00]
 [2.00000000e+00]
 [4.79915819e-04]]


(array([[ 1.99999982e+00, -3.06836096e-08],
        [ 4.00000000e+00,  0.00000000e+00],
        [ 8.00000000e+00,  0.00000000e+00],
        [ 9.99999976e+00, -4.07633895e-08]]), array([[4.79916380e-04],
        [2.00000000e+00],
        [2.00000000e+00],
        [4.79915819e-04]]))

In [168]:
X_12 = [2, 0]
c_1 = [0, 0]
c_2 = [4, 0]
r_1 = 3
r_2 = 3

X_23 = [6, 0]
c_3 = [8, 0]
r_3 = 3

X_34 = [10, 0]
c_4 = [12, 0]
r_4 = 3

args = [{(1,2):X_12, (2,3):X_23, (3,4):X_34}]
init_params = np.array([[3,0,0], [3,4,0], [3,8,0], [4,12,0]])
print(init_params.shape)
x_proj = minimize_projection_loss(init_params, args)
print(x_proj)

(4, 3)
     fun: 4.000001035516371
     jac: array([-2.19857693e-03,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
       -1.68800354e-04,  0.00000000e+00,  4.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00])
 message: 'Optimization terminated successfully.'
    nfev: 99
     nit: 7
    njev: 7
  status: 0
 success: True
       x: array([ 7.54970098e-04,  2.00084715e+00, -7.49712193e-09,  6.06219636e+00,
       -6.82302364e-04,  0.00000000e+00,  2.00000000e+00,  8.00000000e+00,
        0.00000000e+00,  4.00000000e+00,  1.20000000e+01,  0.00000000e+00])
0.0007549700979841115
